## Customer Dataset Cleaning

In [1]:
# %pip install fuzzywuzzy
# !python -m pip install --upgrade pip
# !pip install pycountry
# %pip install python-Levenshtein
# %pip install pandas numpy matplotlib seaborn scikit-learn jupyter

In [2]:
# Activate kernal:
# .\venv_fyp\Scripts\activate

# Step 1: Import libraries
import pandas as pd
import numpy as np
import os
import pycountry
import re
import requests
import time
from datetime import datetime, date
from fuzzywuzzy import process, fuzz

In [3]:
# Step 2: Load dataset
# Replace 'customer_dataset.csv' with your actual file name or path
file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Customer - Copy.csv"
# file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_balanced_5033_rows.csv"
# file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_light_7719_rows.csv"
# file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_stress_6047_rows.csv"

original_customer_dataset_name = "2021 - 2025 Customer - Copy.csv"

# Read dataset
customer_df = pd.read_csv(file_path)

# Show first few rows (original raw data)
customer_df.head()

,CustomerID,Date of Birth,Gender,City,State
0,CUST0001,NaN,NaN,Sungai Besi,Kuala Lumpur
1,CUST0001,NaN,NaN,NaN,NaN
2,CUST0002,NaN,NaN,Mutiara Damansara,Selangor
3,CUST0002,NaN,NaN,NaN,NaN
4,CUST0003,NaN,NaN,Shah Alam,Selangor


### Data Profiling

In [4]:
## Create later

### Perform Data Cleaning Pipeline - CustomerDataset

In [5]:
# ============================================= STAGE 1: SCHEMA & COLUMN VALIDATION =============================================
# # Optional columns
def check_optional_columns(df, threshold=0.8):
    """
    Check optional columns for fill percentage and drop columns that are mostly empty.
    Returns the modified DataFrame and a friendly message.
    """
    print("[LOG] Running check_optional_columns...")
    optional_columns = ["date of birth", "gender"]
    dropped_columns = []

    for col in optional_columns:
        if col in df.columns:
            fill_ratio = df[col].notna().mean()
            print(f"[LOG] Optional column '{col}' fill ratio: {fill_ratio:.2f}")
            if fill_ratio < threshold:
                dropped_columns.append(col)
                df.drop(columns=[col], inplace=True)  # Drop the column immediately
                # df[col].count(): This counts the number of non-missing (non-null/non-NaN) values in the current column (col).
                # len(df): This gives the total number of rows in the DataFrame.
                # fill_ratio: The division calculates the proportion of filled (non-missing) values in that column. A ratio of 1.0 means the column is entirely filled; a ratio of 0.1 means 90% of the values are missing.
                print(f"[LOG] Dropped optional column '{col}' due to too many missing values")
        else:
            print(f"[LOG] Optional column '{col}' not found")

    # Generate user-friendly message
    if dropped_columns:
        dropped_str = ", ".join(dropped_columns)
        message = (
            f"We noticed that very few entries were provided for {dropped_str}. "
            "These columns have been removed. "
            "Segmentation will still be performed using geographic (City, State) "
            "and behavioral data (e.g., orders, purchase items, total spend)."
        )
    else:
        message = "All optional columns have enough data and are kept for analysis."
    
    return df, message

# Mandatory columns 
def check_mandatory_columns(df, threshold=0.8):
    """
    Check mandatory columns for missing values (column-wise).
    Does not drop columns — only warns user if any column is too incomplete.
    Returns the DataFrame and a message summarizing issues.
    """
    print("[LOG] Running check_mandatory_columns...")
    mandatory_columns = ["customerid", "city", "state"]
    missing_report = []
    warning_columns = []

    for col in mandatory_columns:
        if col in df.columns:
            fill_ratio = df[col].notna().mean()
            print(f"[LOG] Mandatory column '{col}' fill ratio: {fill_ratio:.2f}")
            missing_percent = (1 - fill_ratio) * 100
            missing_report.append(f"{col}: {missing_percent:.1f}% missing")

            # Warn if missing exceeds threshold
            if fill_ratio < threshold:
                warning_columns.append(col)
        else:
            print(f"[LOG] Mandatory column '{col}' not found")
            # Handle case where column completely missing
            missing_report.append(f"{col}: column not found (100% missing)")
            warning_columns.append(col)

    # Generate friendly message
    if warning_columns:
        warning_str = ", ".join(warning_columns)
        message = (
            f"Some key fields have a high number of missing values: {warning_str}. "
            "The system will still continue cleaning and processing, "
            "but missing values will be handled automatically by our system. "
            "Please ensure your source data is as complete as possible for more accurate segmentation results.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )
    else:
        message = (
            "All mandatory columns have sufficient data and are ready for cleaning.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )

    return df, message

# ============================================= STAGE 2: REMOVE DUPLICATE ENTRY ROW =================================================
def remove_duplicate_entries(df):
    """Remove duplicate rows, keeping the first occurrence"""
    print("[LOG] Running remove_duplicate_entries...")
    initial_len = len(df)
    df = df.drop_duplicates(keep='first', ignore_index=True)
    print(f"[LOG] Removed {initial_len - len(df)} duplicate rows.")
    return df

# ============================================= STAGE 3: STANDARDIZATION & NORMALIZATION =============================================

def normalize_columns_name(df):
    """Normalize column names: lowercase, strip spaces"""
    print("[LOG] Running normalize_columns_name...")
    df.columns = df.columns.str.strip().str.lower()
    print(f"[LOG] Columns after normalization: {list(df.columns)}")
    return df

# ===============================================================================

def standardize_customer_id(df):
    """Standardize CustomerID format while preserving NaN"""
    print("[LOG] Running standardize_customer_id...")
    if 'customerid' in df.columns:
        # Fill NaN with empty string before converting to string
        df['customerid'] = df['customerid'].fillna('').astype(str).str.strip().str.upper()
        print("[LOG] CustomerID column standardized")
    else:
        print("[LOG] CustomerID column not found, skipping")
    return df
# Might have special case of dirty data exist such as "****", "1234....", "annbwbciwbciowb"
# not sure how to handle it (Currently will say bcs we focus on small business enterprise that have use digital system, so normally customerID will not have inconsistent format issue, even the inconsistant format exist, at the end this row will not be use as when we merge we cant found that customerID)

# ===============================================================================

def standardize_dob(df):
    """Standardize Date of Birth column and convert to YYYY-MM-DD"""
    print("[LOG] Running standardize_dob...")
    # Rename only 'date of birth' to 'dob'
    df = df.rename(columns={'dateofbirth': 'dob'})  
    if 'dob' in df.columns:
        print("[LOG] DOB column found, parsing dates...")
        def parse_date(x):
            if pd.isnull(x):
                return pd.NaT
            for format in ("%d/%m/%Y", "%m-%d-%y", "%Y-%m-%d", "%d-%b-%Y", "%d-%m-%Y"):    
                try:
                    return datetime.strptime(str(x), format).date() # Final format: YYYY-MM-DD | 2025-10-15
                except Exception:
                    continue
            return pd.NaT  # If no valid format found
        df['dob'] = df['dob'].apply(parse_date)
        df['dob'] = pd.to_datetime(df['dob'])
        print("[LOG] DOB parsing complete. Invalid dates marked as NaT")
    else:
        print("[LOG] DOB column not found, skipping")
    return df

# %d/%m/%Y → 12/05/2000
# %m-%d-%y → 05-12-00
# %Y-%m-%d → 2000-05-12
# %d-%b-%Y → 12-May-2000
# %d-%m-%Y → 12-5-2000

# ===============================================================================

def derive_age_features(df):
    """Derive Age from DOB"""
    print("[LOG] Running derive_age_features...")
    if 'dob' in df.columns:
        today = date.today()
        df['age'] = df['dob'].apply(
            lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day))
            if pd.notnull(x) else None
        )
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
        print("[LOG] Age derived from DOB")
    else:
        print("[LOG] DOB column not found, skipping")
    return df
# Example: ((today.month, today.day) < (x.month, x.day))
# (10,15) < (12,1) → True (birthday in Dec is after Oct 15)
# (10,15) < (10,16) → True (birthday tomorrow)
# (10,15) < (5,20) → False (birthday already passed)

# This function calculates each person’s age from their date of birth (dob) by subtracting their birth year from the current year and adjusting if their birthday hasn’t occurred yet this year.

# ===============================================================================

def derive_age_group(df):
    """Derive Age Group based on defined buckets"""
    print("[LOG] Running derive_age_group...")
    if 'age' in df.columns:
        def categorize_age(age):
            if pd.isnull(age):
                return 'Unknown'
            if age < 18: return 'Below 18'
            elif 18 <= age <= 24: return '18-24'
            elif 25 <= age <= 34: return '25-34'
            elif 35 <= age <= 44: return '35-44'
            elif 45 <= age <= 54: return '45-54'
            elif 55 <= age <= 64: return '55-64'
            else: return 'Above 65'
        df['age_group'] = df['age'].apply(categorize_age)
        print("[LOG] Age groups derived")
    else:
        print("[LOG] Age column not found, skipping")
    return df
# ===============================================================================

def drop_dob_after_age_derived(df):
    """Drop DOB column after deriving age and age_group"""
    print("[LOG] Running drop_dob_after_age_derived...")
    if 'dob' in df.columns:
        df = df.drop(columns=['dob'])
        print("[LOG] Dropped DOB column")
    else:
        print("[LOG] DOB column not found, skipping")
    return df

# =================================================================================

def standardize_gender(df):
    """Clean and standardize gender values"""
    print("[LOG] Running standardize_gender...")
    if 'gender' in df.columns:
        df['gender'] = (
            df['gender']
            .astype(str)
            .str.strip()
            .str.lower()
            .replace({
                'm': 'Male', 'male': 'Male', 'man': 'Male', 'boy': 'Male',
                'f': 'Female', 'female': 'Female', 'woman': 'Female', 'girl': 'Female'
            })
        )
        df.loc[~df['gender'].isin(['Male', 'Female']), 'gender'] = 'Unknown'
        print("[LOG] Gender standardized (vectorized)")
    else:
        print("[LOG] Gender column not found, skipping")
    return df

# ==================================================================================

def standardize_location(df):
    """Standardize City, and State fields"""
    print("[LOG] Running standardize_location...")
        
    # Helper function: detect suspicious city names
    def is_suspicious_city(name):
        if not name or name.strip() == '':
            return True
        name = str(name).strip()
        
        if len(name) < 2 or len(name) > 50: # Too short or too long
            return True
        
        if re.search(r'[^A-Za-z\s\'-]', name):  # Contains non-alphabetic or weird symbols   # letters, space, apostrophe, dash allowed
            return True
        
        if re.search(r'(.)\1{3,}', name):   # Repeated characters (e.g., "Ccciiiittty")
            return True
        return False

    # --- City ---
    if 'city' in df.columns:
        df['city'] = df['city'].fillna('').astype(str).str.title().str.strip()
        suspicious_mask = df['city'].apply(lambda x: is_suspicious_city(x) or x.lower() in ['others', 'other'])
        suspicious_count = suspicious_mask.sum()
        df.loc[suspicious_mask, 'city'] = 'Unknown'
        
        print(f"[LOG] Standardized 'city'. Suspicious/unknown entries set to 'Unknown': {suspicious_count}")
    else:
        print("[LOG] 'city' column not found, skipping city standardization")
    
    # --- State ---
    if 'state' in df.columns:
        # malaysia_states = ["Johor", "Kedah", "Kelantan", "Melaka", "Negeri Sembilan","Pahang", "Perak", "Perlis", "Pulau Pinang", "Sabah", "Sarawak", "Selangor", "Terengganu", "Kuala Lumpur", "Labuan", "Putrajaya"]
        malaysia_states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
        alias_map = {
            "Kuala Lumpur": "Wilayah Persekutuan Kuala Lumpur",
            "Kl": "Wilayah Persekutuan Kuala Lumpur",
            "Labuan": "Wilayah Persekutuan Labuan",
            "Putrajaya": "Wilayah Persekutuan Putrajaya"
        }

        df['state'] = df['state'].fillna('').astype(str).str.title().str.strip()
        # ⚡ Match only unique states once
        unique_states = df['state'].unique()
        state_map = {}
        for s in unique_states:
            s_clean = s.strip().title()

            if not s_clean or s_clean == 'Unknown':
                state_map[s] = 'Unknown'

            elif s_clean in alias_map:   # Check alias first
                state_map[s] = alias_map[s_clean]

            else:
                match, score = process.extractOne(s_clean, malaysia_states, scorer=fuzz.token_sort_ratio)
                state_map[s] = match if score >= 80 else 'Unknown'

        # Apply mapping to the dataframe
        df['state'] = df['state'].map(state_map)
        print("[LOG] State standardized (cached fuzzy matching)")
    else:
        print("[LOG] 'state' column not found, skipping state standardization")

    return df

# ============================================= STAGE 4: MISSING VALUE HANDLING =============================================
def handle_missing_values(df):
    print("[LOG] Running handle_missing_values...")

    API_KEY = "68f8ce9a38c3f632237334dyiedb96e"
    GEOCODE_URL = "https://geocode.maps.co/search"
    SLEEP_TIME = 1.2
    cache = {}  # ⚡ moved outside loops

    # --- Drop rows without ID ---
    if 'customerid' in df.columns:
        before_drop = len(df)
        df = df[df['customerid'].notna()].copy()
        print(f"[LOG] Dropped {before_drop - len(df)} rows without CustomerID")
    else:
        print("[LOG] 'customerid' column missing, skipping drop")

    # ----- Age -----
    if 'age' in df.columns:
        missing_ratio = df['age'].isna().mean()
        print(f"[LOG] Age missing ratio: {missing_ratio:.2%}")
        if missing_ratio > 0:
            if 'gender' in df.columns and df['gender'].nunique() > 1:
                # Group by gender if available
                df['age'] = df.groupby('gender')['age'].transform(
                    lambda x: x.fillna(x.median())
                )
                print("[LOG] Applied gender-based median imputation for missing age")
                
                # Always fill any remaining missing values with overall median 
                df['age'] = df['age'].fillna(df['age'].median())
                
                # Update derived column if needed
                df = derive_age_group(df)
        else:
            print("[LOG] No missing values for age")
    else:
        print("[LOG] 'age' column missing, skipping age imputation")

    # --- Gender mode imputation ---
    if 'gender' in df.columns:
        mode = df['gender'].mode()[0]
        unknown_mask = df['gender'] == 'Unknown'
        count = unknown_mask.sum()
        df.loc[unknown_mask, 'gender'] = mode
        print(f"[LOG] Replaced {count} 'Unknown' gender values with mode: {mode}")
    else:
        print("[LOG] 'gender' column missing, skipping gender imputation")
        
   
    # --- City & State handling ---
    if {'city', 'state'}.issubset(df.columns):
        print("\n🔍 Handling missing city/state values...")

        # malaysia_states = ["Johor", "Kedah", "Kelantan", "Melaka", "Negeri Sembilan","Pahang", "Perak", "Perlis", "Pulau Pinang", "Sabah", "Sarawak", "Selangor", "Terengganu", "Kuala Lumpur", "Labuan", "Putrajaya"]
        malaysia_states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
        cache = {}  # city -> validated state
        SLEEP_TIME = 1.2
        # Case 1: missing state but city known → fill via geocoding API
        print("\n[LOG] Case 1: Filling missing state where city is known...")
        # Get rows needing state fill (city known, state unknown)
        mask_case1 = (df['city'] != 'Unknown') & (df['state'] == 'Unknown')
        cities_to_query = df.loc[mask_case1, 'city'].unique().tolist()

        print(f"[LOG] {len(cities_to_query)} unique cities need state lookup")
        
        for city in cities_to_query:
            if city not in cache:
                # Call API
                try:
                    resp = requests.get(GEOCODE_URL, params={"q": f"{city}, Malaysia", "api_key": API_KEY}, timeout=10)
                    if resp.status_code == 200:
                        data = resp.json()
                        if isinstance(data, list) and data:
                            state_name = data[0].get("address", {}).get("state")
                            if state_name and state_name in malaysia_states:
                                cache[city] = state_name
                            else:
                                cache[city] = None
                        else:
                            cache[city] = None
                    else:
                        cache[city] = None
                    time.sleep(SLEEP_TIME)
                except Exception as e:
                    print(f"[WARN] Failed to get state for city '{city}': {e}")
                    cache[city] = None

            # Fill values
            fill_state = cache.get(city)
            if fill_state:
                df.loc[(df['city'] == city) & (df['state'] == 'Unknown'), 'state'] = fill_state
                print(f"[TRACE] Filled {city} → state='{fill_state}' (API valid)")
            else:
                # Fallback: use mode state & mode city for that state
                valid_states = df[df['state'] != 'Unknown']['state']
                mode_state = valid_states.mode()[0] if not valid_states.empty else 'Unknown'

                # Compute mode city per state
                mode_city_per_state = (
                    df[df['city'] != 'Unknown'].groupby('state')['city']
                    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
                    .to_dict()
                )
                mode_city = mode_city_per_state.get(mode_state, 'Unknown')

                mask_fill = (df['city'] == city) & (df['state'] == 'Unknown')
                df.loc[mask_fill, 'state'] = mode_state
                df.loc[mask_fill, 'city'] = mode_city
                print(f"[TRACE] Filled {mask_fill.sum()} row(s) → city='{mode_city}', state='{mode_state}' (Fallback)")

        # Case 2: missing city but state known → fill with mode city per state
        print("\n[LOG] Case 2: Filling missing city where state is known...")
        mask_case2 = (df['city'] == 'Unknown') & (df['state'] != 'Unknown')
        if mask_case2.any():
            mode_city_per_state = (
                df[df['city'] != 'Unknown'].groupby('state')['city']
                .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
                .to_dict()
            )
            for state, city_mode in mode_city_per_state.items():
                mask_fill = mask_case2 & (df['state'] == state)
                df.loc[mask_fill, 'city'] = city_mode
                print(f"[TRACE] Filled {mask_fill.sum()} row(s) → missing city for state='{state}' → city='{city_mode}'")

        # Case 3: both missing → fill with most frequent pair
        print("\n[LOG] Case 3: Filling missing city and state...")
        mask_case3 = (df['city'] == 'Unknown') & (df['state'] == 'Unknown')
        if mask_case3.any():
            valid_pairs = df[(df['city'] != 'Unknown') & (df['state'] != 'Unknown')]
            if not valid_pairs.empty:
                city_mode, state_mode = valid_pairs.groupby(['city', 'state']).size().idxmax()
                df.loc[mask_case3, ['city', 'state']] = [city_mode, state_mode]
                print(f"[TRACE] Filled {mask_case3.sum()} row(s) → missing city/state → City='{city_mode}', State='{state_mode}'")
            else:
                print("[WARN] No valid city/state pairs to fill missing both values")

    return df

# ============================================= STAGE 5: OUTLIER DETECTION =============================================

def detect_outliers(df):
    """Adaptive outlier handling based on dataset size."""
    print("[LOG] Running detect_outliers...")
    if 'age' in df.columns:
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
        # df['age_original'] = df['age']  # ✅ Keep a copy of original age values (for comparison or re-deriving age_group)

        n = len(df)
        print(f"[LOG] Dataset has {n} rows")
        if n < 500:
            # IQR method
            Q1 = df['age'].quantile(0.25)
            Q3 = df['age'].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = df[(df['age'] <  Q1) | (df['age'] > Q3)].shape[0]
            df.loc[(df['age'] < lower_bound) | (df['age'] > upper_bound), 'age'] = np.nan
            print(f"[LOG] IQR Applied for {n} rows. Range: [{lower_bound:.1f}, {upper_bound:.1f}] Outliers set to NaN: {outliers}")
        else:
            # Percentile capping
            lower_bound = df['age'].quantile(0.01)
            upper_bound = df['age'].quantile(0.99)
            df['age'] = df['age'].clip(lower=lower_bound, upper=upper_bound)
            print(f"[LOG] Percentile capping  applied for {n} rows. Capped to [{lower_bound:.1f}, {upper_bound:.1f}]")
    else:
        print("[LOG] 'age' column missing, skipping outlier detection")
    return df

def deduplicate_customers(df):
    print("[LOG] Running deduplicate_customers...")
    if 'customerid' not in df.columns:
        print("[LOG] 'customerid' column missing, skipping deduplication")
        return df

    def resolve_conflict(series):
        vals = series.dropna().unique()
        if len(vals) == 0:
            return pd.NA
        elif len(vals) == 1:
            return vals[0]
        else:
            return series.mode().iloc[0]

    # ⚡ Vectorized groupby instead of per-group loop
    df = df.groupby('customerid', as_index=False).agg(resolve_conflict)
    print("[LOG] Deduplication complete (vectorized)")
    return df


In [6]:
def clean_customer_dataset(df, original_customer_dataset_name):
    """
    Main cleaning pipeline for customer dataset.
    Executes all stages in proper order:
    0. Column Normalization
    1. Schema & Column Validation
    2. Duplicate Entry Removal
    3. Standardization & Normalization
    4. Missing Value Handling
    5. Outlier Detection
    6. Deduplication
    Finally, saves the cleaned dataset and returns it.
    """
    print("🚀 Starting data cleaning pipeline...\n")

    # =============================================
    # STAGE 0: NORMALIZE COLUMN NAMES
    # =============================================
    print("========== [STAGE 0 START] Normalize Column Names ==========")
    df = normalize_columns_name(df)
    print("✅ [STAGE 0 COMPLETE] Column names normalized.\n")

    # =============================================
    # STAGE 1: SCHEMA & COLUMN VALIDATION
    # =============================================
    print("========== [STAGE 1 START] Schema & Column Validation ==========")
    df, optional_msg = check_optional_columns(df)
    df, mandatory_msg = check_mandatory_columns(df)
    print(optional_msg)
    print(mandatory_msg)
    print("✅ [STAGE 1 COMPLETE] Schema validation done.\n")

    # =============================================
    # STAGE 2: REMOVE DUPLICATE ENTRY ROWS
    # =============================================
    print("========== [STAGE 2 START] Remove Duplicate Entry Rows ==========")
    df = remove_duplicate_entries(df)
    print("✅ [STAGE 2 COMPLETE] Duplicate entries removed.\n")

    # =============================================
    # STAGE 3: DEDUPLICATION
    # =============================================
    print("========== [STAGE 3 START] Deduplication ==========")
    df = deduplicate_customers(df)
    print("✅ [STAGE 3 COMPLETE] Duplicate CustomerIDs deduplicated.\n")

    # =============================================
    # STAGE 4: STANDARDIZATION & NORMALIZATION
    # =============================================
    print("========== [STAGE 3 START] Standardization & Normalization ==========")
    df = standardize_customer_id(df)
    df = standardize_dob(df)
    df = derive_age_features(df)
    df = derive_age_group(df)
    df = drop_dob_after_age_derived(df)
    df = standardize_gender(df)
    df = standardize_location(df)
    print("✅ [STAGE 3 COMPLETE] Standardization and normalization finished.\n")
    
    print("========== [Standardized Dataset START] Save Standardized Dataset ==========")
    base_name, ext = os.path.splitext(original_customer_dataset_name)
    cleaned_file = f"{base_name}_standardized{ext}"
    df.to_csv(cleaned_file, index=False)
    print(f"✅ [FINAL STAGE COMPLETE] Cleaned dataset saved as: {cleaned_file}\n")
    
    # =============================================
    # STAGE 5: MISSING VALUE HANDLING
    # =============================================
    print("========== [STAGE 4 START] Missing Value Handling ==========")
    df = handle_missing_values(df)
    print("✅ [STAGE 4 COMPLETE] Missing values handled.\n")

    # # =============================================
    # # STAGE 6: OUTLIER DETECTION
    # # =============================================
    print("========== [STAGE 5 START] Outlier Detection ==========")
    df = detect_outliers(df)   # make sure detect_outliers returns df
    print("✅ [STAGE 5 COMPLETE] Outliers handled.\n")

    # =============================================
    # SAVE CLEANED DATASET
    # =============================================
    print("========== [FINAL STAGE START] Save Cleaned Dataset ==========")
    base_name, ext = os.path.splitext(original_customer_dataset_name)
    cleaned_file = f"{base_name}_cleaned{ext}"
    df.to_csv(cleaned_file, index=False)
    print(f"✅ [FINAL STAGE COMPLETE] Cleaned dataset saved as: {cleaned_file}\n")

    print("==========================================================")
    print("🎉 Data cleaning pipeline completed successfully!\n")
    return df, cleaned_file

In [7]:
# Keep a copy for comparison
original_customer_df = customer_df.copy()
cleaned_customer_df, cleaned_file_name = clean_customer_dataset(customer_df, original_customer_dataset_name)
cleaned_customer_df.head()

🚀 Starting data cleaning pipeline...

========== [STAGE 0 START] Normalize Column Names ==========
[LOG] Running normalize_columns_name...
[LOG] Columns after normalization: ['customerid', 'date of birth', 'gender', 'city', 'state']
✅ [STAGE 0 COMPLETE] Column names normalized.

========== [STAGE 1 START] Schema & Column Validation ==========
[LOG] Running check_optional_columns...
[LOG] Optional column 'date of birth' fill ratio: 0.00
[LOG] Dropped optional column 'date of birth' due to too many missing values
[LOG] Optional column 'gender' fill ratio: 0.00
[LOG] Dropped optional column 'gender' due to too many missing values
[LOG] Running check_mandatory_columns...
[LOG] Mandatory column 'customerid' fill ratio: 1.00
[LOG] Mandatory column 'city' fill ratio: 0.55
[LOG] Mandatory column 'state' fill ratio: 0.53
We noticed that very few entries were provided for date of birth, gender. These columns have been removed. Segmentation will still be performed using geographic (City, State) a

,customerid,city,state
0,CUST0001,Sungai Besi,Wilayah Persekutuan Kuala Lumpur
1,CUST0002,Mutiara Damansara,Selangor
2,CUST0003,Shah Alam,Selangor
3,CUST0004,Muar,Johor
4,CUST0005,Petaling Jaya,Selangor


In [8]:
def generate_cleaning_report(original_df, cleaned_df):
    report = {}

    # Missing values
    report['missing_before'] = original_df.isna().sum()
    report['missing_after'] = cleaned_df.isna().sum()

    # Duplicate count
    report['duplicates_before'] = len(original_df) - len(original_df.drop_duplicates())
    report['duplicates_after'] = len(cleaned_df) - len(cleaned_df.drop_duplicates())

    # Gender consistency
    if 'gender' in original_df.columns:
        report['unique_genders_before'] = original_df['gender'].nunique()
        report['unique_genders_after'] = cleaned_df['gender'].nunique()

    # Example summary
    summary = pd.DataFrame(report)
    summary.to_csv("cleaning_summary.csv")
    print(summary)
    return summary

# ===============================================
# LAST STAGE : GENERATE CLEANING SUMMARY REPORT 
# ===============================================
summary = generate_cleaning_report(original_customer_df, cleaned_customer_df)

               missing_before  missing_after  duplicates_before  \
City                   2588.0            NaN                879   
CustomerID                0.0            NaN                879   
Date of Birth          5775.0            NaN                879   
Gender                 5775.0            NaN                879   
State                  2706.0            NaN                879   
city                      NaN            0.0                879   
customerid                NaN            0.0                879   
state                     NaN            0.0                879   

               duplicates_after  
City                          0  
CustomerID                    0  
Date of Birth                 0  
Gender                        0  
State                         0  
city                          0  
customerid                    0  
state                         0  
